In [159]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
import dash_table
import pandas as pd
import json

In [160]:
# Build App
app = JupyterDash(__name__,suppress_callback_exceptions=True)

#function to query dbpedia endpoint
def sql(value):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(value)
    sparql.setReturnFormat(JSON)
    result = sparql.query()
    #file = list() #create an empty list
    #loop to put each of the individual records in a new row
    #for result in results:
    #    file.append(result)       
    #return file   
    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)


app.layout = html.Div([
    
    html.Div(
        className="navbar",
        children=[ 
            html.H3(
                children="Interactive Data Analytics Dashboard for RDF Knowledge Graphs", 
                className="navbar--title"),
            html.Div(
                children=[
                dcc.Tabs(id='tabs', value='Dashboard', children=[
                    dcc.Tab(label='Dashboard', value='Dashboard'),
                    dcc.Tab(label='Tables', value='Tables'),
                    dcc.Tab(label='Charts', value='Charts'),
                    dcc.Tab(label='Query Samples', value='Samples'),
                    dcc.Tab(label='Upload', value='Upload'),
                    dcc.Tab(label='Download', value='Download'),
                ]),
                html.Div(id='tabs-content')
                ])
    ]),
   
       
])#lay-out div    

#Query examples
data = {'Query Examples':  ['First example', 'Second example','third example']}
df = pd.DataFrame (data, columns = ['Query Examples'])
ResultListdataframe = pd.DataFrame (data=[], columns = [])

@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value'),
)

def render_content(tab):
    if tab == 'Dashboard':
        return html.Div([
            html.Div(
                className="querybox",
                children=[
                    html.H4(
                        className="querybox--title",
                        children="SPARQL Query"
                    ),
                    dcc.Textarea(
                        id= "query-text", 
                        value="",
                        placeholder="Enter your SPARQL query.",
                        className="querybox--textarea"
                    ),
                    html.Button(
                        id="submit-btn", className="querybox--btn", 
                        children="SUBMIT", 
                        n_clicks=0
                    )
            ]),
            html.Div(
                id='MainTableDiv',
                children=[
                    dash_table.DataTable(
                        id='Resulttable',
                        style_table={'width': '800px'},
                        style_cell=dict(textAlign='left'),   
                        page_current=0,
                        page_size=10,
                        page_action='custom',
                    )
                ],
                style= {'display': 'none'}
            ),
            html.Div(
                className="result-graph",
                children=[
                    dcc.Graph(id='graph')
                ]
            )
             
        ])
    elif tab == 'Tables':
        return html.Div([
            html.H3('Tables')
        ])
    elif tab == 'Charts':
        return html.Div([
            html.H3('Charts')
        ])
    elif tab == 'Samples':
        return html.Div([
            html.H3('Query Samples'),
            html.Div(
                className="querybox",
                children=[
                    html.H4(
                        className="querybox--title",
                        children="SPARQL Query"
                    ),
                    dcc.Textarea(
                        id= "query-text", 
                        value="",
                        placeholder="Enter your SPARQL query.",
                        className="querybox--textarea"
                    ),
                    html.Button(
                        id="submit-btn", className="querybox--btn", 
                        children="SUBMIT", 
                        n_clicks=0
                    )
            ]),
            html.Div(
                className="examples",
                children=[
                    dash_table.DataTable(
                    id='table',
                    columns=[{"name": i, "id": i} for i in df.columns],
                    data=df.to_dict('records'),
                    style_table={'width': '800px'},
                    style_cell=dict(textAlign='left'),                   
                    )
                ]),
        ])
    
@app.callback(
    [Output("Resulttable", "data"), Output('Resulttable', 'columns'), Output('MainTableDiv', 'style')],
    [Input("submit-btn", "n_clicks"),
    Input('Resulttable', "page_current"),
    Input('Resulttable', "page_size")],
    [State("query-text","value")]
)    
def update_output(n_clicks,page_current,page_size, value):
    if n_clicks > 0:
        resultList = sql(value)
        ResultListdataframe = resultList
        mycolumns = [{'name': index, 'id': index} for index in resultList.columns]
        print(mycolumns)   
        return resultList[
        page_current*page_size:(page_current+ 1)*page_size
    ].to_dict('records'),mycolumns,{'display': 'block'}
    return [],[],{'display': 'none'}

@app.callback(
    Output("graph", "figure"),
    Input("submit-btn", "n_clicks"),
    State("query-text","value")
)
def gen_graph(n_clicks, value):
    df = [{'x': [1, 2, 3] }] 
    fig = px.scatter(df)
    if n_clicks > 0:
        resultList = sql(value)
        ResultListdataframe = resultList
        print(ResultListdataframe)
        fig = px.scatter(ResultListdataframe)
#         fig = px.bar(ResultListdataframe)
#         fig = px.line(ResultListdataframe)

    return fig


In [161]:
# Run app and display result inline in the notebook
#app.run_server(mode='inline')

# Run app and display result on localhost
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
[{'name': 'author_name', 'id': 'author_name'}, {'name': 'title', 'id': 'title'}]
                    author_name                            title
0                 Abbie Hoffman                  Steal This Book
1                  Alice Walker                 The Color Purple
2                    Barry Took                  Round the Horne
3              Charles Beaumont                The Twilight Zone
4                 David Eddings                    The Belgariad
5                 David Eddings                    The Malloreon
6                  Edward Albee  Who's Afraid of Virginia Woolf?
7                     Greg Bear              Blood Music (novel)
8                   Victor Hugo      The Hunchback of Notre-Dame
9               William Golding                Lord of the Flies
10  William Makepeace Thackeray              Vanity Fair (novel)
11              Subin Bhattarai                          Monsoon
12                Betty Fussell